# gamma(d)

In [1]:
using DifferentialEquations
using Plots
using LinearAlgebra
using Distributions  # Подключаем для функции Uniform

# Параметры системы
N = 5
R = ones(N)

Delta = 0.001
omega0 = 3.0
#omega = rand(Uniform(omega0 - Delta, omega0 + Delta), N) # собственные частоты
#phi_0 = rand(Uniform(0, 2 * π), N)

omega=[3.0000401969406063, 3.0004776479947406, 3.000743856462235, 3.0000850916493222, 3.0009747133631475]
phi_0=[0.8684839852505457, 2.782384121591725, 4.0786232496029, 2.098936954402864, 0.9451386520541617]

println("начальное phi=", phi_0)
println("начальное omega=", omega, "\n")

T_0 = 2000 # начало анализа
T = 5000 # конец анализа

gamma_values = range(0, π, length=40)
d_values = range(0, 1.0, length=40)

epsilon = 1e-4 # Точность для проверки синхронизации

function kuramoto!(dphi, phi, p, t)
    d, gamma, omega, R = p
    N = length(phi)
    for j in 1:N
        sum_term = 0.0
        for k in 1:N
            if j != k
                sum_term += R[k] * sin(phi[k] - phi[j] - gamma)
            end
        end
        dphi[j] = omega[j] + (d / (N * R[j])) * sum_term
    end
end

# Списки для хранения точек и их синхронизации
d_points = Float64[]
gamma_points = Float64[]
sync_status = Int[] # 1, если синхронизация, иначе 0

# Основной цикл по сетке параметров
for gamma in gamma_values
    for d in d_values
        p = (d, gamma, omega, R)
        
        # Интеграция уравнений движения
        prob = ODEProblem(kuramoto!, phi_0, (0.0, T), p)
        sol = solve(prob, saveat=[T_0, T], abstol=1e-10, reltol=1e-8)

        # Получение фаз на T_0 и T
        phi_T0, phi_T = sol.u[1], sol.u[2]

        # Средние наблюдаемые частоты для всех осцилляторов
        avg_frequencies = (phi_T .- phi_T0) / (T - T_0)

        # Проверка на синхронизацию
        if maximum(avg_frequencies) - minimum(avg_frequencies) < epsilon
            sync = 1 # Синхронизированы
        else
            sync = 0 # Нет синхронизации
        end

        # Сохраняем результаты
        push!(d_points, d)
        push!(gamma_points, gamma)
        push!(sync_status, sync)
    end
end



начальное phi=[0.8684839852505457, 2.782384121591725, 4.0786232496029, 2.098936954402864, 0.9451386520541617]
начальное omega=[3.0000401969406063, 3.0004776479947406, 3.000743856462235, 3.0000850916493222, 3.0009747133631475]



In [2]:
gr(fmt=:png, dpi=600) 

scatter(
    d_points, gamma_points, 
    group=sync_status,  
    color=["yellow" "purple"],  
    label=["Нет синхронизации" "Синхронизация"],  
    xlabel="d", ylabel="γ", 
    title="N=$N, ω=$omega0, Δ=$Delta, ε=$epsilon (точность совпадения)",
    grid=true, legend=:topright,
    size=(800, 600),  
    titlefontsize=12,  
    guidefontsize=10,  
    tickfontsize=8,
    gridalpha=0.3,  # Прозрачность сетки
    gridwidth=0.5,  # Толщина линий сетки
    tickpadding=5,  # Увеличиваем отступ между метками и осями
    xticks=(0:0.1:1),  # Мелкие метки по оси X
    yticks=(0:0.5:5),  # Мелкие метки по оси Y
    framestyle=:box,  # Добавляем рамку вокруг графика
    tickdirection=:in  # Направление меток внутрь
      
)

# Убедимся, что папка `results` существует
mkpath("results")

# Сохранение графика
savefig("results/sync_plot.png")

"g:\\odeint\\Pazo_model\\Julia\\results\\sync_plot.png"

# частота(d)

In [3]:
using DifferentialEquations
using Plots
using LinearAlgebra
using Distributions  # Для функции Uniform
using Printf

# Параметры системы
N = 5
R = ones(N)

Delta = 0.001
omega0 = 3.0

# Собственные частоты и начальные фазы
omega = [3.0000401969406063, 3.0004776479947406, 3.000743856462235, 3.0000850916493222, 3.0009747133631475]
phi_0 = [0.8684839852505457, 2.782384121591725, 4.0786232496029, 2.098936954402864, 0.9451386520541617]

println("Начальное phi=", phi_0)
println("Начальное omega=", omega, "\n")

# Параметры времени
T_0 = 2000 # Начало анализа
T = 5000   # Конец анализа

# Значения параметров γ и d
#gamma_values = [3.1415926]
#d_values = [1.0]

gamma_values = range(0, π, length=40)
d_values = range(0, 1.0, length=40)

# Определение системы уравнений Курамото
function kuramoto!(dphi, phi, p, t)
    d, gamma, omega, R = p
    N = length(phi)
    for j in 1:N
        sum_term = 0.0
        for k in 1:N
            if j != k
                sum_term += R[k] * sin(phi[k] - phi[j] - gamma)
            end
        end
        dphi[j] = omega[j] + (d / (N * R[j])) * sum_term
    end
end

# Основной цикл по сетке параметров для анализа синхронизации
for gamma in gamma_values
    for d in d_values
        p = (d, gamma, omega, R)
        
        # Интеграция уравнений движения
        prob = ODEProblem(kuramoto!, phi_0, (0.0, T), p)
        sol = solve(prob, saveat=0.1:0.2:T, abstol=1e-10, reltol=1e-8)

        # Преобразуем решения в матрицу для удобства
        phi_vals = hcat(sol.u...)  # Каждая колонка — фазы в момент времени
        times = sol.t

        # Мгновенные частоты
        frequencies = diff(phi_vals, dims=2) ./ diff(times)'

        # Средние частоты для каждого осциллятора
        mean_frequencies = [(phi_vals[i, end] - phi_vals[i, findfirst(t -> t >= T_0, times)]) / (T - T_0) for i in 1:N]

        display(frequencies)
        display(mean_frequencies)

        # Построение графиков
        p = plot()  # Создаем пустой график

        # Мгновенные частоты для каждого осциллятора
        for i in 1:N
            plot!(p, times[2:end], frequencies[i, :], xlabel="Время", ylabel="Частота", 
                label="Осциллятор $i (Мгновенная частота)", linewidth=2, xlim=(0, 100))
        end

        # Добавление средних частот для каждого осциллятора
        for i in 1:N
            plot!(p, times[2:end], fill(mean_frequencies[i], length(times)-1), 
                label="Осциллятор $i (Средняя частота)", linestyle=:dash, linewidth=2,
                title="gamma=$(@sprintf("%.5f", gamma)) d=$(@sprintf("%.5f", d))",
                size=(800, 600),  
                titlefontsize=12,  
                guidefontsize=10,  
                tickfontsize=8,
                gridalpha=0.3,          # Прозрачность сетки
                gridwidth=0.5,          # Толщина линий сетки
                tickpadding=5,          # Увеличиваем отступ между метками и осями
                xticks=(0:5.0:100),     # Мелкие метки по оси X
                yticks=(0:0.1:7),       # Мелкие метки по оси Y
                framestyle=:box,        # Добавляем рамку вокруг графика
                tickdirection=:in       # Направление меток внутрь
                )
        end

        mkpath("results")
        # Сохранение графика
        savefig("results/freq_gamma=$(@sprintf("%.5f", gamma)) d=$(@sprintf("%.5f", d)).png")


    end
end



5×24999 Matrix{Float64}:
 3.00004  3.00004  3.00004  3.00004  …  3.00004  3.00004  3.00004  3.00004
 3.00048  3.00048  3.00048  3.00048     3.00048  3.00048  3.00048  3.00048
 3.00074  3.00074  3.00074  3.00074     3.00074  3.00074  3.00074  3.00074
 3.00009  3.00009  3.00009  3.00009     3.00009  3.00009  3.00009  3.00009
 3.00097  3.00097  3.00097  3.00097     3.00097  3.00097  3.00097  3.00097

5-element Vector{Float64}:
 2.9998401942608495
 3.0002776161515796
 3.0005438068718426
 2.9998850859765844
 3.0007746483822957

5×24999 Matrix{Float64}:
 3.00977  3.00979  3.00981  3.00983  …  3.00046  3.00046  3.00046  3.00046
 2.99239  2.99239  2.99238  2.99237     3.00046  3.00046  3.00046  3.00046
 2.99137  2.99133  2.99129  2.99125     3.00046  3.00046  3.00046  3.00046
 2.99851  2.99852  2.99853  2.99853     3.00046  3.00046  3.00046  3.00046
 3.01028  3.0103   3.01032  3.01033     3.00046  3.00046  3.00046  3.00046

5-element Vector{Float64}:
 3.0002642670954227
 3.0002642701316935
 3.000264278655078
 3.000264266853226
 3.00026426890762

5×24999 Matrix{Float64}:
 3.01954  3.01962  3.01971  3.01979  …  3.00046  3.00046  3.00046  3.00046
 2.9843   2.98427  2.98425  2.98423     3.00046  3.00046  3.00046  3.00046
 2.98192  2.98175  2.98158  2.98142     3.00046  3.00046  3.00046  3.00046
 2.99695  2.99698  2.99701  2.99704     3.00046  3.00046  3.00046  3.00046
 3.01962  3.0197   3.01977  3.01985     3.00046  3.00046  3.00046  3.00046

...

5×24999 Matrix{Float64}:
 2.66223  2.69293  2.72264  2.75085  …  3.0003   3.0003   3.0003   3.0003
 3.29584  3.27991  3.26089  3.24013     3.00035  3.00035  3.00035  3.00035
 3.29956  3.25096  3.20959  3.17479     3.00091  3.00091  3.00091  3.00091
 3.06778  3.07294  3.07569  3.07639     3.00004  3.00004  3.00004  3.00004
 2.67691  2.70559  2.73351  2.76016     3.00072  3.00072  3.00072  3.00072

5-element Vector{Float64}:
 2.999971990815802
 3.000150586402325
 3.0006761252855156
 2.9998687900587466
 3.0006538590878877

5×24999 Matrix{Float64}:
 2.65417  2.68648  2.71768  2.74723  …  3.0003   3.0003   3.0003   3.0003
 3.30323  3.28629  3.2661   3.24413     3.00035  3.00035  3.00035  3.00035
 3.30603  3.25515  3.21205  3.17599     3.00091  3.00091  3.00091  3.00091
 3.06973  3.07506  3.0778   3.07837     3.00004  3.00004  3.00004  3.00004
 2.66916  2.69934  2.72868  2.75661     3.00072  3.00072  3.00072  3.00072

5-element Vector{Float64}:
 2.9999719951854105
 3.0001505880641175
 3.00067612511366
 2.9998687813662
 3.0006538618561875

Начальное phi=[0.8684839852505457, 2.782384121591725, 4.0786232496029, 2.098936954402864, 0.9451386520541617]
Начальное omega=[3.0000401969406063, 3.0004776479947406, 3.000743856462235, 3.0000850916493222, 3.0009747133631475]

